In [ ]:
import numpy as np
import pandas as pd
import ast
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [ ]:
movies = pd.read_csv('./files/movies_metadata.csv', low_memory=False)
keyword = pd.read_csv('./files/keywords.csv')
credits = pd.read_csv('./files/credits.csv')


In [ ]:
def getCast(text):
  counter = 0
  res = []
  for i in ast.literal_eval(text):
    counter+=1
    if(counter > 5):
      break
    res.append(i['name'])

  return res

In [ ]:
def getDirector(text):
  res = []
  for i in ast.literal_eval(text):
    if(i['job'] == 'Director'):
      res.append(i['name'])
  return res

In [ ]:
def convert(text):
  res = []
  for i in ast.literal_eval(text):
    res.append(i['name'])
  
  return res

In [ ]:
def collapse(text):
    res = []
    for i in text:
        res.append(i.replace(" ",""))
    return res

In [ ]:
credits['crew'] = credits['crew'].apply(getDirector)
credits['cast'] = credits['cast'].apply(getCast)

In [ ]:
credits.head(1)

In [ ]:
movies = movies[['genres', 'id', 'imdb_id', 'title', 'overview']]
movies['genres'] = movies['genres'].apply(convert)
keyword['keywords'] = keyword['keywords'].apply(convert)

In [ ]:
movies['id'] = pd.to_numeric(movies['id'], errors='coerce')
movies=movies.dropna(subset=['id'])
movies['id'] = movies['id'].astype(int)

In [ ]:
movies = movies.merge(keyword, on='id')
movies = movies.merge(credits, on='id')

In [ ]:
movies.info()

In [ ]:
movies['cast'] = movies['cast'].apply(collapse)
movies['crew'] = movies['crew'].apply(collapse)
movies['genres'] = movies['genres'].apply(collapse)

In [ ]:
movies['overview'] = movies['overview'].astype(str)
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [ ]:
movies['title'] = movies['title'].astype(str)
movies['title'] = movies['title'].apply(lambda x:x.split())

In [ ]:
movies['tags'] = movies['genres'] + movies['title'] + movies['overview'] + movies['keywords'] + movies['crew'] + movies['cast']
df = movies.drop(columns=['genres', 'overview', 'keywords', 'crew', 'cast'])

In [ ]:
df['tags']

In [ ]:
df['tags'] = df['tags'].apply(lambda x: " ".join(x))
df['title'] = df['title'].apply(lambda x: " ".join(x))

In [ ]:
df.head()

In [ ]:
cv = CountVectorizer(max_features=5000, stop_words='english')

In [ ]:
vector = cv.fit_transform(df['tags']).toarray()

In [ ]:
similarity = cosine_similarity(vector)

In [ ]:
df[df['title'] == 'The Lego Movie'].index[0]

In [ ]:
def recommend(movie):
    index = df[df['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(df.iloc[i[0]].title)

In [ ]:
pickle.dump(df,open('movie_list.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))